# Production-Ready Agent Workflow with Flotorch and Autogen

This notebook demonstrates a comprehensive, production-style workflow. It integrates all the advanced features we've explored into a single, robust application:
- **Remote Agent Configuration**: Uses `FlotorchAutogenAgent` to load agent personas from the Flotorch UI.
- **Custom Local Tools**: Defines and attaches a custom tool to the agent at runtime.
- **Dual Memory System**: Implements both short-term session memory and long-term external memory.

### Prerequesit
Configure agent, memory provider and API key in Flotroch console (https://console.flotorch.cloud/)

### Viewing logs
Logs can be viewed in logs tab in Flotroch console (https://console.flotorch.cloud/)

## 1. Setup and Imports

The following cells install dependencies, set API credentials, and import all necessary components for handling tools, memory, and remote agent management.

In [ ]:
# install flotorch autogen package
%pip install flotorch[autogen]

In [ ]:
FLOTORCH_API_KEY = "<flotorch api key>"
FLOTORCH_BASE_URL = "<flotroch gateway base url>" # eg: https://gateway.flotorch.cloud"
MEMORY_PROVIDER = "<flotorch memory provider>"
AGENT_NAME = "<flotorch agent name>"
USER_ID = "flotorch_user_001"
APP_ID = "flotorch_app_001"

In [ ]:
# Import necessary libraries
from flotorch.autogen.agent import FlotorchAutogenAgent
from flotorch.autogen.sessions import FlotorchAutogenSession
from flotorch.autogen.memory import FlotorchAutogenMemory

from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_core.tools import FunctionTool
from autogen_core.memory import MemoryContent, MemoryMimeType


from autogen_agentchat.ui import Console

print("Imported necessary libraries successfully")

## 2. Custom Tool Definition and Memory Configuration

Here, we define a custom `get_weather` tool locally using Autogen's `FunctionTool`. This tool can be dynamically attached to our agent at runtime, demonstrating how to extend a centrally-managed agent with new capabilities. We also configure the full dual-memory system: `FlotorchAutogenSession` for conversational context and `FlotorchAutogenMemory` for persistent, long-term knowledge.

In [ ]:
def get_weather(city: str, units: str = "imperial") -> str:
    
        if units == "imperial":
            return f"The weather in {city} is 96 °F and Rainy."
        elif units == "metric":
            return f"The weather in {city} is 23 °C and Sunny."
        else:
            return f"Sorry, I don't know the weather in {city}."

tools = [FunctionTool(get_weather, "Get the weather for a given city.")]

print("Custom tool defined successfully.")

In [ ]:
session_memory = FlotorchAutogenSession(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL
)


external_memory = FlotorchAutogenMemory(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    name = MEMORY_PROVIDER,
    user_id = USER_ID,
    app_id  = APP_ID
)

print("Initialized session_memory and external_memory")

## 3. Agent Initialization with Custom Tools and Memory

We initialize the `FlotorchAutogenAgent` client, loading our agent's configuration from the Flotorch UI. Crucially, we also pass our locally defined `tools` into the `custom_tools` parameter and configure both `model_context` (for short-term memory) and `memory` (for long-term memory), augmenting the agent's capabilities.

In [ ]:
flotorch_client = FlotorchAutogenAgent(
    agent_name =AGENT_NAME,
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    custom_tools = tools,
    memory = [external_memory],
    model_context = session_memory,

)

agent = flotorch_client.get_agent()

print("Fetched agent is initialized")

## 4. Assembling the Complete Team

Finally, we assemble the `RoundRobinGroupChat`, bringing together the remotely-configured agent, its custom tool, and both memory modules. This creates a fully-featured, production-ready agent capable of advanced, context-aware interactions.

In [ ]:
team = RoundRobinGroupChat(
        participants=[agent],
        max_turns=1
    )
print(f"Agent and Task created successfully: {agent.name}")

## 5. Interactive Demonstration

Engage with the agent in this interactive loop to test its full range of capabilities. Try asking it to get the weather for a city (to test the tool) and then ask it to recall information from the conversation. Note that user inputs and agent responses are manually added to the long-term memory after each turn. Type 'exit' to end the session.

In [ ]:
while True:
    user_input = input("user: ")

    if user_input.lower().strip() == "exit":
        break
    
    # Use this for normal agent.
    stream = team.run_stream(task = user_input)
    
    # Use this if you configured output schema to the agent.
    stream = agent.run_stream(task = user_input)
    result = await Console(stream)
     # --- Add user input to memory ---
    await external_memory.add(MemoryContent(content=user_input, mime_type=MemoryMimeType.TEXT))

    # --- Add assistant response(s) to memory ---
    if hasattr(result, "messages"):
        for msg in result.messages[-1]:
            # Only save text messages (type == "TextMessage")
            if getattr(msg, "type", None) == "TextMessage":
                
                await external_memory.add(MemoryContent(content=getattr(msg, "content", ""), mime_type=MemoryMimeType.TEXT))


## Summary

This notebook served as a **capstone project**, successfully integrating all previously demonstrated concepts into a single, comprehensive, and production-ready workflow.  
We built an advanced AI agent that combines the power of **remote configuration**, **local custom tools**, and a **full dual-memory system**.

### Key Achievements

- **Hybrid Configuration** 
  Initialized an agent using `FlotorchAutogenAgent` to load its base configuration from the Flotorch UI, then augmented it at runtime with locally defined custom tools.

- **Full Memory Stack** 
  Equipped the agent with both `FlotorchAutogenSession` for conversational flow and `FlotorchAutogenMemory` for long-term knowledge persistence.

- **End-to-End Demonstration** 
  Showcased the agent's ability to seamlessly switch between using its custom tools for specific tasks and accessing its memory to provide contextually rich, intelligent responses.

- **Robust Architecture** 
  Delivered a complete workflow that serves as a powerful and flexible blueprint for building sophisticated, real-world AI applications.